# TP 06 V2 : Fine-tuning GPT-2 - Version Optimisée

**Objectif** : Fine-tuner GPT-2 français pour générer des descriptions de Pokémon

**Nouveautés V2** :
- Option pour ajouter les noms de Pokémon au vocabulaire (smart initialization)
- Option pour figer les couches basses (faster training)
- Meilleur filtrage des données

**Durée** : 2h

---

## 0. Installation et imports

In [ ]:
# Installation des dépendances (Colab)
!pip install -q transformers datasets accelerate

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from datasets import load_dataset
import random
import warnings
warnings.filterwarnings('ignore')

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

# Reproductibilité
torch.manual_seed(42)
random.seed(42)

---

## 1. Configuration des options d'entraînement

### Nouvelles options V2

In [ ]:
# ═══════════════════════════════════════════════════════════════
#                    OPTIONS D'ENTRAÎNEMENT V2
# ═══════════════════════════════════════════════════════════════

# Option 1 : Ajouter les noms de Pokémon au vocabulaire
# Les nouveaux tokens seront initialisés avec l'embedding de "Pokémon" ou "animal"
# Avantage : Le modèle sait déjà que "Pikachu" est un Pokémon
ADD_POKEMON_TOKENS = True

# Option 2 : Figer les couches basses du modèle
# Seules les couches hautes seront entraînées
# Avantage : Plus rapide, moins d'overfitting
FREEZE_LOWER_LAYERS = True
NUM_LAYERS_TO_FREEZE = 6  # Sur 12 couches, figer les 6 premières

# Autres paramètres
MAX_SAMPLES = 5000      # Nombre d'articles pour le fine-tuning
MAX_LENGTH = 256        # Longueur max des séquences
NUM_EPOCHS = 3          # Nombre d'epochs

print("Configuration V2 :")
print(f"  - Ajouter tokens Pokémon : {ADD_POKEMON_TOKENS}")
print(f"  - Figer couches basses : {FREEZE_LOWER_LAYERS} ({NUM_LAYERS_TO_FREEZE} couches)")
print(f"  - Samples : {MAX_SAMPLES}")
print(f"  - Epochs : {NUM_EPOCHS}")

---

## 2. Charger GPT-2 français

In [ ]:
# Charger le tokenizer et le modèle
model_name = "asi/gpt-fr-cased-small"

print("Chargement du tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(model_name)

print("Chargement du modèle...")
model = AutoModelForCausalLM.from_pretrained(model_name)

print(f"\n✅ Modèle chargé !")
print(f"Paramètres : {sum(p.numel() for p in model.parameters()):,}")
print(f"Vocabulaire initial : {len(tokenizer):,} tokens")
print(f"Nombre de couches : {model.config.n_layer}")

In [ ]:
# Ajouter un token de padding
# GPT-2 n'a pas de token PAD natif, on réutilise EOS
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.pad_token_id

---

## 3. Option 1 : Ajouter les tokens Pokémon (Smart Initialization)

### Pourquoi ?
- "Pikachu" est tokenisé en plusieurs sous-tokens par BPE
- En ajoutant "Pikachu" comme token unique, initialisé avec "Pokémon", le modèle sait déjà que c'est un Pokémon
- Apprentissage plus rapide et meilleure qualité

In [ ]:
# Liste des noms de Pokémon à ajouter
# On charge depuis le dataset pokemon-names-fr
print("Chargement des noms de Pokémon...")
names_dataset = load_dataset("chris-lmd/pokemon-names-fr")
POKEMON_NAMES = [item["name"] for item in names_dataset["train"]]
print(f"  {len(POKEMON_NAMES)} noms chargés")
print(f"  Exemples : {POKEMON_NAMES[:10]}")

In [ ]:
def add_pokemon_tokens_to_vocab(tokenizer, model, pokemon_names):
    """
    Ajoute les noms de Pokémon au vocabulaire et initialise leurs embeddings.
    
    Les nouveaux tokens sont initialisés avec l'embedding de "Pokémon" ou "animal"
    pour que le modèle sache déjà qu'il s'agit de créatures.
    """
    # Trouver le token de référence pour l'initialisation
    # On essaie "Pokémon", sinon "Pokemon", sinon "animal"
    reference_tokens = ["Pokémon", "Pokemon", "animal", "créature"]
    reference_id = None
    reference_word = None
    
    for ref in reference_tokens:
        tokens = tokenizer.encode(ref, add_special_tokens=False)
        if len(tokens) == 1:  # Token unique trouvé
            reference_id = tokens[0]
            reference_word = ref
            break
    
    if reference_id is None:
        # Fallback : utiliser le premier token de "Pokémon"
        reference_id = tokenizer.encode("Pokémon", add_special_tokens=False)[0]
        reference_word = "Pokémon (premier sous-token)"
    
    print(f"Token de référence : '{reference_word}' (id={reference_id})")
    
    # Filtrer : garder seulement les noms qui ne sont pas déjà des tokens uniques
    new_tokens = []
    for name in pokemon_names:
        tokens = tokenizer.encode(name, add_special_tokens=False)
        if len(tokens) > 1:  # Le nom est découpé en plusieurs tokens
            new_tokens.append(name)
    
    print(f"Tokens à ajouter : {len(new_tokens)} (sur {len(pokemon_names)})")
    
    if len(new_tokens) == 0:
        print("Aucun nouveau token à ajouter.")
        return 0
    
    # Sauvegarder l'embedding de référence AVANT le resize
    with torch.no_grad():
        reference_embedding = model.transformer.wte.weight[reference_id].clone()
    
    # Ajouter les tokens au vocabulaire
    num_added = tokenizer.add_tokens(new_tokens)
    print(f"Tokens ajoutés au vocabulaire : {num_added}")
    
    # Redimensionner les embeddings du modèle
    old_size = model.transformer.wte.weight.shape[0]
    model.resize_token_embeddings(len(tokenizer))
    new_size = model.transformer.wte.weight.shape[0]
    print(f"Embeddings : {old_size:,} → {new_size:,}")
    
    # Initialiser les nouveaux tokens avec l'embedding de référence
    with torch.no_grad():
        for i in range(num_added):
            new_token_id = old_size + i
            # Copier l'embedding de référence + petit bruit pour différencier
            noise = torch.randn_like(reference_embedding) * 0.01
            model.transformer.wte.weight[new_token_id] = reference_embedding + noise
    
    print(f"✅ {num_added} tokens initialisés avec l'embedding de '{reference_word}'")
    return num_added

In [ ]:
# Appliquer si l'option est activée
if ADD_POKEMON_TOKENS:
    print("═" * 50)
    print("Ajout des tokens Pokémon au vocabulaire...")
    print("═" * 50)
    num_added = add_pokemon_tokens_to_vocab(tokenizer, model, POKEMON_NAMES)
    print()
    
    # Vérification
    test_name = "Pikachu"
    tokens = tokenizer.encode(test_name, add_special_tokens=False)
    print(f"Test : '{test_name}' → {tokens} ({len(tokens)} token(s))")
else:
    print("Option ADD_POKEMON_TOKENS désactivée.")

---

## 4. Option 2 : Figer les couches basses (Partial Freezing)

### Pourquoi ?
- Les couches basses capturent des features générales (grammaire, syntaxe)
- Ces features sont déjà bien apprises par le pré-entraînement
- On ne fine-tune que les couches hautes (sémantique, style)

### Avantages
- Entraînement plus rapide (moins de gradients)
- Moins d'overfitting
- Préserve les connaissances linguistiques

In [ ]:
def freeze_lower_layers(model, num_layers_to_freeze):
    """
    Fige les embeddings et les N premières couches du transformer.
    
    Args:
        model: Le modèle GPT-2
        num_layers_to_freeze: Nombre de couches à figer (depuis le bas)
    """
    total_layers = model.config.n_layer
    
    if num_layers_to_freeze >= total_layers:
        print(f"⚠️ Attention : vous figez {num_layers_to_freeze} couches sur {total_layers} !")
        num_layers_to_freeze = total_layers - 1
    
    # Figer les embeddings (tokens et positions)
    for param in model.transformer.wte.parameters():
        param.requires_grad = False
    for param in model.transformer.wpe.parameters():
        param.requires_grad = False
    
    # Figer les N premières couches
    for i in range(num_layers_to_freeze):
        for param in model.transformer.h[i].parameters():
            param.requires_grad = False
    
    # Compter les paramètres
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    frozen_params = total_params - trainable_params
    
    print(f"Couches figées : {num_layers_to_freeze} / {total_layers}")
    print(f"Paramètres totaux : {total_params:,}")
    print(f"Paramètres figés : {frozen_params:,} ({100*frozen_params/total_params:.1f}%)")
    print(f"Paramètres entraînables : {trainable_params:,} ({100*trainable_params/total_params:.1f}%)")
    
    return trainable_params

In [ ]:
# Appliquer si l'option est activée
if FREEZE_LOWER_LAYERS:
    print("═" * 50)
    print("Freezing des couches basses...")
    print("═" * 50)
    trainable = freeze_lower_layers(model, NUM_LAYERS_TO_FREEZE)
else:
    print("Option FREEZE_LOWER_LAYERS désactivée.")
    print("Tous les paramètres seront entraînés (full fine-tuning).")

In [ ]:
# Déplacer le modèle sur GPU
model = model.to(device)
print(f"\nModèle déplacé sur : {device}")

---

## 5. Test du modèle avant fine-tuning

In [ ]:
def generate_text(prompt, max_length=100, temperature=0.8, top_k=50):
    """Génère du texte à partir d'un prompt."""
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs.input_ids.to(device)
    attention_mask = inputs.attention_mask.to(device)
    
    with torch.no_grad():
        outputs = model.generate(
            input_ids,
            attention_mask=attention_mask,
            max_length=max_length,
            temperature=temperature,
            top_k=top_k,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id
        )
    
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
# Test avant fine-tuning
prompt = "Pikachu est un Pokémon de type"
print(f"Prompt: {prompt}")
print(f"Génération (AVANT fine-tuning):")
print(generate_text(prompt))

---

## 6. Préparer le dataset Pokémon

In [ ]:
# Charger le dataset
print("Chargement du dataset Pokémon...")
dataset = load_dataset("chris-lmd/pokepedia-fr")

print(f"\n✅ Dataset chargé !")
print(f"Articles : {len(dataset['train']):,}")
print(f"Colonnes : {dataset['train'].column_names}")

In [ ]:
# Aperçu d'un article
sample = dataset['train'][0]
print(f"Titre: {sample.get('title', 'N/A')}")
print(f"Types: {sample.get('types', [])}")
print(f"\nContenu (500 premiers caractères):")
print(sample['content'][:500] + "...")

In [ ]:
# Meilleur filtre V2 : utiliser le champ 'types'
# Les vrais Pokémon ont au moins un type
def is_pokemon_description(example):
    """Filtre les articles décrivant des Pokémon."""
    # Critère 1 : a des types (c'est un vrai Pokémon)
    has_types = len(example.get('types', [])) > 0
    
    # Critère 2 : contient des patterns de description
    text = example['content'].lower()
    has_patterns = any(p in text for p in ["est un pokémon", "évolution", "capacité"])
    
    return has_types or has_patterns

# Filtrer
filtered_dataset = dataset['train'].filter(is_pokemon_description)
print(f"Articles après filtrage : {len(filtered_dataset):,}")

In [ ]:
# Limiter le nombre de samples
if len(filtered_dataset) > MAX_SAMPLES:
    train_dataset = filtered_dataset.shuffle(seed=42).select(range(MAX_SAMPLES))
else:
    train_dataset = filtered_dataset

print(f"Échantillon pour fine-tuning : {len(train_dataset):,} articles")

In [ ]:
# Tokenization
def tokenize_function(examples):
    """Tokenize les textes avec troncature."""
    return tokenizer(
        examples['content'],
        truncation=True,
        max_length=MAX_LENGTH,
        padding='max_length'
    )

print("Tokenization en cours...")
tokenized_dataset = train_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=train_dataset.column_names
)

# Ajouter les labels
def add_labels(examples):
    examples['labels'] = examples['input_ids'].copy()
    return examples

tokenized_dataset = tokenized_dataset.map(add_labels, batched=True)
print(f"✅ Tokenization terminée !")
print(f"Colonnes : {tokenized_dataset.column_names}")

---

## 7. Fine-tuning

In [ ]:
# Configuration de l'entraînement
training_args = TrainingArguments(
    output_dir="./gpt2-pokemon-v2",
    overwrite_output_dir=True,
    
    # Hyperparamètres
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    
    # Optimiseur
    learning_rate=5e-5,
    warmup_steps=100,
    weight_decay=0.01,
    
    # Logging
    logging_steps=50,
    save_steps=500,
    save_total_limit=2,
    
    # Misc
    fp16=torch.cuda.is_available(),
    report_to="none",
)

print("Configuration :")
print(f"  - Epochs : {training_args.num_train_epochs}")
print(f"  - Batch size effectif : {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"  - Learning rate : {training_args.learning_rate}")
print(f"  - FP16 : {training_args.fp16}")

In [ ]:
# Créer le Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

print("✅ Trainer créé !")

In [ ]:
# Résumé de la configuration
print("═" * 60)
print("           RÉSUMÉ DE LA CONFIGURATION V2")
print("═" * 60)
print(f"\n📊 Dataset : {len(train_dataset):,} articles")
print(f"📝 Max length : {MAX_LENGTH} tokens")
print(f"🔄 Epochs : {NUM_EPOCHS}")
print(f"\n🎯 Options :")
print(f"   - Tokens Pokémon ajoutés : {ADD_POKEMON_TOKENS}")
print(f"   - Couches figées : {FREEZE_LOWER_LAYERS} ({NUM_LAYERS_TO_FREEZE if FREEZE_LOWER_LAYERS else 0}/{model.config.n_layer})")
print(f"\n💾 Vocabulaire : {len(tokenizer):,} tokens")

trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total = sum(p.numel() for p in model.parameters())
print(f"🧠 Paramètres entraînables : {trainable:,} / {total:,} ({100*trainable/total:.1f}%)")
print("═" * 60)

In [ ]:
# Lancer le fine-tuning
print("🚀 Fine-tuning V2 en cours...")
print()

trainer.train()

print("\n✅ Fine-tuning terminé !")

In [ ]:
# Sauvegarder le modèle
trainer.save_model("./gpt2-pokemon-v2-final")
tokenizer.save_pretrained("./gpt2-pokemon-v2-final")
print("✅ Modèle V2 sauvegardé !")

---

## 8. Génération de descriptions

In [ ]:
def generate_description(prompt, max_length=150, temperature=0.8, top_k=50, top_p=0.9):
    """Génère une description de Pokémon."""
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs.input_ids.to(device)
    attention_mask = inputs.attention_mask.to(device)
    
    with torch.no_grad():
        outputs = model.generate(
            input_ids,
            attention_mask=attention_mask,
            max_length=max_length,
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id,
            repetition_penalty=1.2,
            no_repeat_ngram_size=3
        )
    
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
# Test : Pikachu
prompt = "Pikachu est un Pokémon de type"
print(f"Prompt: {prompt}")
print(f"\nGénération (APRÈS fine-tuning V2):")
print(generate_description(prompt))

In [ ]:
# Test : Pokémon inventé
prompt = "Flamador est un Pokémon de type Feu. Il"
print(f"Prompt: {prompt}")
print(f"\nGénération:")
print(generate_description(prompt))

In [ ]:
# Test : Évolution
prompt = "Dracaufeu évolue à partir de"
print(f"Prompt: {prompt}")
print(f"\nGénération:")
print(generate_description(prompt))

---

## 9. Comparaison des configurations

Pour comparer l'effet des options, relancez le notebook avec différentes configurations :

| Configuration | ADD_POKEMON_TOKENS | FREEZE_LOWER_LAYERS | Résultat attendu |
|---------------|-------------------|---------------------|------------------|
| Baseline | False | False | Référence |
| + Tokens | True | False | Meilleure connaissance des noms |
| + Freeze | False | True | Plus rapide, moins d'overfitting |
| Full V2 | True | True | Combinaison des deux |

---

## 10. Récapitulatif V2

### Nouvelles techniques apprises

1. **Smart Token Initialization** : Ajouter des tokens au vocabulaire et les initialiser intelligemment
2. **Partial Freezing** : Figer les couches basses pour un entraînement plus efficace
3. **Meilleur filtrage** : Utiliser les métadonnées (types) pour sélectionner les bons articles

### Quand utiliser ces techniques ?

| Technique | Quand l'utiliser |
|-----------|------------------|
| **Add tokens** | Vocabulaire spécialisé (noms propres, termes techniques) |
| **Freeze layers** | Petit dataset, risque d'overfitting, GPU limité |
| **Full fine-tuning** | Grand dataset, GPU puissant, besoin de flexibilité |